In [1]:
import torch
from torch import nn, Tensor
import numpy as np
import random
from torchvision import models
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import v2 as transforms_v2
from torchvision.transforms import ToTensor
from torchvision.io import decode_image
import torch.optim.lr_scheduler as lr_scheduler
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional
import time
from PIL import Image
from datetime import datetime
from torchvision.models import resnet18, ResNet18_Weights

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using {device} device")

Using mps device


In [ ]:
# Define the model and local path for saving weights
model_resnet_18 = models.resnet18()
weights_path = "./output/resnet18_weights.pth"

# Check if the file exists locally
if os.path.exists(weights_path):
    print("✅ Loading existing ResNet-18 weights...")
    state_dict = torch.load(weights_path, map_location="cpu")
    model_resnet_18.load_state_dict(state_dict)
else:
    print("⬇️  Downloading ResNet-18 weights...")
    model_resnet_18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # Downloads pretrained weights
    torch.save(model_resnet_18.state_dict(), weights_path)
    print("💾 Weights saved locally at:", weights_path)

In [4]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [5]:

dataset_paths = ["/Users/leonardomassaro/Desktop/DatasetOxford_prof/", "/scratch.hpc/leomass/ipcv-assignment-2/dataset/", "/scratch.hpc/leonardo.massaro2/ipcv-assignment-2/dataset/"]
correct_path = None

for path in dataset_paths:
    if os.path.exists(path) and os.path.isdir(path):
        print("Detected dataset on ", path)
        correct_path = path
if not correct_path:
    raise Exception("No dataset found")

Detected dataset on  /Users/leonardomassaro/Desktop/DatasetOxford_prof/


In [6]:
class OxfordPetDataset(Dataset):
    def __init__(self, path, file_name, transform=None) -> None:
        super().__init__()
        self.filename = file_name
        self.root = Path(path)
        self.transform = transform
        self.names, self.labels = self._get_names_and_labels()
        tot_size_bytes = 0
        n_images = 0
        loaded_data = []
        idx = 0
        while idx < len(self.names):
            name = self.names[idx]
            img_tensor = self.get_img_from_filesystem(name)
            tot_size_bytes += img_tensor.numel() * img_tensor.element_size()
            n_images += 1
            loaded_data.append(img_tensor)
            idx += 1
        print("tot size", tot_size_bytes, "bytes, for", n_images, "images")
        self.data_tensor = torch.stack(loaded_data).to(device=device)
        self.labels = torch.Tensor(self.labels).type(torch.LongTensor).to(device=device)
        
    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:

        label = self.labels[idx]
        img = self.data_tensor[idx]
        img = self.transform(img).to(device)

        return img, label
    
    def get_img_from_filesystem(self, name) -> Tensor:
        img_path = self.root / "images" / f"{name}.jpg"
        img = Image.open(img_path).convert("RGB")
        img = transforms_v2.Resize((256,256))(img)
        img = transforms_v2.ToTensor()(img)
        return img


    def get_num_classes(self) -> int:
        return max(self.labels) + 1

    def _get_names_and_labels(self) -> Tuple[List[str], List[int]]:
        names = []
        labels = []

        with open(self.root / "annotations" / self.filename) as f:
            for line in f:
                if(line[0] == "#"):
                    continue
                name, label = line.replace("\n", "").split(" ")
                names.append(name),
                labels.append(int(label) - 1)

        return names, labels

In [7]:
def train(dataloader, model, loss_fn, optimizer, scheduler, lr_data):
    model.train()
    total_loss = 0.0
    num_batches = 0

    for X, y in dataloader:
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        num_batches += 1
        lr_data.append(optimizer.param_groups[0]["lr"])

    avg_loss = total_loss / num_batches
    return avg_loss

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accuracy = 100*correct
    return accuracy, test_loss

In [9]:

def evaluate_model(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.append(preds)
            all_labels.append(labels)

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # --- Accuracy ---
    accuracy = correct / total

    # --- Confusion Matrix ---
    num_classes = outputs.shape[1]
    conf_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)
    for t, p in zip(all_labels, all_preds):
        conf_matrix[t, p] += 1

    # --- Precision, Recall, F1 ---
    tp = conf_matrix.diag()
    fp = conf_matrix.sum(dim=0) - tp
    fn = conf_matrix.sum(dim=1) - tp

    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + fn + 1e-12)
    f1 = 2 * precision * recall / (precision + recall + 1e-12)

    # Weighted (by class support)
    support = conf_matrix.sum(dim=1)
    weighted_precision = (precision * support).sum() / support.sum()
    weighted_recall = (recall * support).sum() / support.sum()
    weighted_f1 = (f1 * support).sum() / support.sum()

    return {
        "test_accuracy": accuracy,
        "test_precision": weighted_precision.item(),
        "test_recall": weighted_recall.item(),
        "test_f1": weighted_f1.item()
    }

In [10]:
def run_experiment(parameters):

    BATCH_SIZE = parameters["batch_size"]
    N_DENS_BLKS = parameters["n_dense_blocks"]
    N_CONVS_EACH_BLCK = parameters["n_conv_each_block"]
    GRWTH_RATE = parameters["growth_rate"]
    DROPOUT = parameters["dropout"]
    BASE_LR = parameters["base_learning_rate"]
    WARMUP_ITERS = parameters["warmup_iter"]
    MAIN_SCH_KICK = parameters["main_sched_epochs_kick_in"]
    EPOCHS = parameters["n_epochs"]

    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

    training_transform_stack = transforms_v2.Compose([
        transforms_v2.RandomResizedCrop(size=(224, 224)),
        transforms_v2.RandomHorizontalFlip(p=0.5),
        transforms_v2.ColorJitter(                               
            brightness=0.5, 
            contrast=0.5, 
            saturation=0.5, 
            hue=0.1
        ),
        transforms_v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1.0)), 
        transforms_v2.ToDtype(torch.float32, scale=True),
        transforms_v2.Normalize(mean=MEAN,
                    std=STD),
    ])

    validation_transform_stack = transforms_v2.Compose([
        transforms_v2.Resize(256),
        transforms_v2.CenterCrop(224),
        transforms_v2.Normalize(MEAN, STD)
    ])

    train_dataset = OxfordPetDataset(correct_path, file_name="train.txt" , transform=training_transform_stack)
    validation_dataset = OxfordPetDataset(correct_path, file_name="val.txt" , transform=validation_transform_stack)
    test_dataset = OxfordPetDataset(correct_path, file_name="test.txt" , transform=validation_transform_stack)

    # Create data loaders.
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # -----------
    num_features = model_resnet_18.fc.in_features
    model_resnet_18.fc = nn.Linear(num_features, 37)  

    # Freeze everything
    for param in model_resnet_18.parameters():
        param.requires_grad = False

    # Unfreeze only the head
    for param in model_resnet_18.fc.parameters():
        param.requires_grad = True

    # -----------

    model = model_resnet_18.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=BASE_LR)

    warmup_scheduler = lr_scheduler.LinearLR(optimizer, start_factor=0.01, end_factor=1., total_iters=WARMUP_ITERS)
    main_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=15)

    batch_lrs = []           # record learning rate per batch
    epoch_end_indices = []   # store index (batch count) at the end of each epoch

    net_performance_data = {
        "train_loss": [],
        "val_loss": [],
        "val_accuracy": []
    }

    t1 = time.time()

    for t in range(EPOCHS):
        net_performance_data["train_loss"].append( train(train_dataloader, model, loss_fn, optimizer, warmup_scheduler, batch_lrs) )
        val_accuracy, val_loss = test(validation_dataloader, model, loss_fn)
        if t > MAIN_SCH_KICK: main_scheduler.step(val_accuracy)
        net_performance_data["val_loss"].append( val_loss )
        net_performance_data["val_accuracy"].append( val_accuracy )
        epoch_end_indices.append(len(batch_lrs))

    t2 = time.time()
    elapsed_time = t2-t1

    print("Done in ", elapsed_time, " sec")

    print("Best accuracy ", np.max(net_performance_data["val_accuracy"]))

    net_performance_data_df = pd.DataFrame(data=net_performance_data)
    
    #add all experiment parameters to the result dataframe
    for key in parameters:
        if isinstance(parameters[key], list):
            for i, el in enumerate(parameters[key]):
                net_performance_data_df[key + "_"+ str(i)] = el
        else:
            net_performance_data_df[key] = parameters[key]
    
    # add elapsed time
    net_performance_data_df["elapsed_time_sec"] = elapsed_time

    # create a column for epochs steps indexes
    net_performance_data_df = net_performance_data_df.reset_index().rename(columns={"index":"epoch"})

    # add final evaluation stats
    eval_performaces = evaluate_model(test_dataloader, model)
    for metric in eval_performaces:
        net_performance_data_df[metric] = eval_performaces[metric]

    return net_performance_data_df


In [11]:
parameters_base = [
{
    "experiment_name": "early_lr_sched_change",
    "batch_size" : 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 3500,
    "main_sched_epochs_kick_in": 40,
    "n_epochs": 17
},

]

In [ ]:
total_results = []

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

### Actual run of the experiments
for par in parameters_base:
    total_results.append(run_experiment(par))
    # update partial results after each experiment
    df_combined = pd.concat(total_results, ignore_index=True)
    df_combined["val_accuracy"].plot()
    plt.show()
    #df_combined.to_csv(f"./output/results_{timestamp}.csv")